## SQL Queries with Differentially Private Set Union (DPSU)

In the context of the WhiteNoise SQL scenario, consider a guiding example: there exists a dataset with Reddit users and their posts, keeping in mind that a single user could have multiple posts. An analyst would like to release a report containing the counts of the number of bigrams per user while preserving privacy. We'd like to increase the representation of bigrams contained in this report without violating privacy bounds.

You can imagine that it might be easy to identify an author's bigrams if she has only posted a few times and therefore has very few bigrams. To resolve this threat, previously, we would have to drop bigrams whose noisy counts were below a certain threshold. However, this process was wasteful because we allocated privacy budget to all rows uniformly even if their counts already exceeded the threshold.

With DPSU, we resolve this issue by adding noise in a dependent fashion. This increases the dimensionality of the final dataset, and we therefore have less data loss and get a richer representation of bigrams. 

The code below showcases DPSU support with pandas.

In [3]:
import pandas as pd
from opendp.whitenoise.metadata import CollectionMetadata
from opendp.whitenoise.sql import PrivateReader, PandasReader

reddit = pd.read_csv("data/readers/reddit.csv", index_col=0)
metadata = CollectionMetadata.from_file("data/readers/reddit.yaml")

query = "SELECT ngram, COUNT(*) as n FROM reddit.reddit GROUP BY ngram"

reader = PandasReader(metadata, reddit)
private_reader = PrivateReader(metadata, reader)

result = private_reader.execute_typed(query)
print(result)

 ngram        |n      
 -------------|-------
  (do, n)     | 47    
  (i, m)      | 71    
  (in, the)   | 40    
  (it, s)     | 54    
  (n, t)      | 111   
  (of, the)   | 44    
  (that, s)   | 46    
  (this, is)  | 35    
  (you, re)   | 36    


### SQL Module with DPSU

In [4]:
from opendp.whitenoise.client import get_execution_client

execution_client = get_execution_client()

project = {"params": {"dataset_name": "reddit", 
                      "budget": 0.5,
                      "query": "SELECT ngram, COUNT(*) as c FROM reddit.reddit GROUP BY ngram"},
           "uri": "modules/sql-module"}

result = execution_client.submit(params=project["params"],
                            uri=project["uri"])

In [7]:
print(result.result)

{"ngram": {"0": "(i, m)", "1": "(i, ve)", "2": "(it, s)", "3": "(n, t)", "4": "(that, s)"}, "c": {"0": 91, "1": 81, "2": 76, "3": 144, "4": 89}}
